In [22]:
from dotenv import load_dotenv
load_dotenv()
import nest_asyncio
nest_asyncio.apply()
from typing import List
from pydantic import BaseModel
import os
import json
import csv

from llama_parse import LlamaParse
from llama_index.core.schema import Document
from llama_index.llms.openai import OpenAI

In [23]:
## Parse the Form Filing File
def parse_file(file_path: str) -> List[Document]:
    llama_parse = LlamaParse(
        api_key=os.environ['LLAMA_CLOUD_API_KEY'],
        result_type='markdown',
        target_pages="0"
    )
    result = llama_parse.load_data(
        file_path,
    )
    return result

In [24]:
# Parsing the report template
documents = parse_file('data/Report_format_2.xlsx')

Started parsing the file under job_id fe08477a-aba7-45ad-808f-d5685e64386c


In [25]:
text = documents[0].text

In [26]:
## Structured Extraction
prompt = f"""
You are an AI assistant specializing in Industrial Engineering problem solving. You've been given an Excel spreadsheet containing items to be improved. \ 
and improvement parameters. Your task is to extract and structure this information in a clear, organized format.

The Excel sheet contains the following:
1. Items to be improved (rows)
2. Description (columns)
3. Improvement parameters and dates (columns)

Input Excel data:
{text}

Please present the extracted and structured information in a clear, easy-to-read format.
"""

In [27]:
## Defining the database
class ReportParameters(BaseModel):
    """Data model for IE problem solving analysis."""
    ItemsToBeImproved: List[str]
    Description: List[str]
    ImprovementParameters: List[str]

In [28]:
## Defining the LLM model
llm = OpenAI(model='gpt-4o-mini')

In [29]:
from llama_index.core.llms import ChatMessage
sllm = llm.as_structured_llm(output_cls=ReportParameters)
input_msg = ChatMessage.from_str(prompt)

In [30]:
## Get the parsed documents as a 
output = sllm.chat([input_msg])
output_obj = output.raw
output_obj

ReportParameters(ItemsToBeImproved=['The saturation of the lower glass point Xiaoli Pill is 65%', 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%', 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%', 'The upper and lower glass bonding saturation is 67.5%', 'Five-in-one front&back saturation53.2%', 'CellstickFPCsaturation61.9%', 'FPCFoam resistance test saturation40.8%', 'The saturation of amplified pills and sports cars is 73.3%', 'pointUVglue,UVsolidify&Check saturation33.3%', 'Electrophoresis tank noteBufferliquid saturation79.2%', 'BufferLiquid injection port sealing and solidification+Water leakage test saturation60.3%', 'Silicone oil saturation 50.7%', 'Silicone oil port sealing + dispensing + curing saturation 58%', 'Set Conn & lower cover + paste QR CODE saturation 41%', 'Lock the lid + label the saturation 81.7%', 'Glass module test saturation 100%', 'Visual inspection of UV glue + pull tape is the bottlen

In [31]:
ItemsToBeImproved = output_obj.ItemsToBeImproved
Description = output_obj.Description
ImprovementParameters = output_obj.ImprovementParameters[1:]

In [32]:
## Config at LLamaIndex
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
# pip install llama-index-indices-managed-llama-cloud
index = LlamaCloudIndex(
  name="objective-wildfowl-2024-11-24", 
  project_name="Default",
  organization_id="2033a7fc-187e-48e4-a172-5079c4ee2bbf",
  api_key=os.environ['LLAMA_CLOUD_API_KEY']
)
query_engine = index.as_query_engine(
    dense_similarity_top_k=10,
    sparse_similarity_top_k=10,
    alpha=0.5,
    enable_reranking=True,
    rerank_top_n=5,
)

In [33]:
## Developing the prompt based on description and generate improvement suggestions accordingly

from tqdm import tqdm
def generate_answers(ItemsToBeImproved: List[str],Description: List[str], ImprovementParameters: List[str]) -> List[str]:
    items_to_be_improved = {}
    for index, item in enumerate(ItemsToBeImproved[:3]):
        items_to_be_improved[item] = {}
        for parameter in tqdm(ImprovementParameters):
            job_saturation = (
                f"Analyze the following text and determine whether the job saturation value mentioned is lower than 95%.\n\n"
                f"Text: \"{Description[index]}\"\n\n"
                f"If a job saturation value is explicitly mentioned, check if it is lower than 95%. If so, respond with 'YES'. "
                f"If it is 95% or higher, respond with 'NO'. If no job saturation value is mentioned, respond with 'NO INFORMATION'."
            )
            input_msg = ChatMessage.from_str(job_saturation)
            output = llm.chat([input_msg])
            answer = output.message.content
            print(Description[index], answer)
            if answer == 'YES':
                query = (
                    f"For the problem '{item}', provide a detailed and concise value or description for the improvement parameter '{parameter}'.\n"
                    f"If the parameter is not applicable or no information is available, respond explicitly with 'NA'.\n\n"
                    f"Ensure your response is clear, contextually relevant, and avoids ambiguity."
                    )
                answer = str(query_engine.query(query))
                items_to_be_improved[item]['Description'] = Description[index]
                items_to_be_improved[item][parameter] = answer
    return items_to_be_improved

In [34]:
answers = generate_answers(ItemsToBeImproved, Description, ImprovementParameters)

 ... (more hidden) ...

Low job saturation(lower than95%) YES


 ... (more hidden) ...

Low job saturation(lower than95%) YES


 ... (more hidden) ...

...

 ... (more hidden) ...

Low job saturation(lower than95%) YES


 ... (more hidden) ...

Low job saturation(lower than95%) YES


 ... (more hidden) ...


In [35]:
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)
# Flatten the nested dictionary
flat_data = []
for parameter, metrics in answers.items():
    flat_metrics = flatten_dict(metrics)
    flat_metrics['Items to be improved'] = parameter 
    flat_data.append(flat_metrics)

# Get all unique keys to use as CSV headers
headers = set()
for item in flat_data:
    headers.update(item.keys())

# Sort headers to ensure 'Items to be improved' comes first
headers = sorted(headers)
headers.insert(0, headers.pop(headers.index('Items to be improved')))
headers.insert(1, headers.pop(headers.index('Description')))
headers.insert(2, headers.pop(headers.index('Improvement direction')))
headers.insert(3, headers.pop(headers.index('Person responsible')))
headers.insert(4, headers.pop(headers.index('Expected start date')))
headers.insert(5, headers.pop(headers.index('Actual start date')))
headers.insert(6, headers.pop(headers.index('Expected completion date')))
headers.insert(7, headers.pop(headers.index('Actual completion date')))
headers.insert(8, headers.pop(headers.index('Improve immediately')))
headers.insert(9, headers.pop(headers.index('Confirmation')))
headers.insert(10, headers.pop(headers.index('appendix')))

In [36]:
import os
os.getcwd()
os.listdir('./data')

['893F8420',
 'BP_Excel.xlsx',
 'gantt_chart_with_custom_xaxis.png',
 'Paper-2.pdf',
 'policy.pdf',
 'PPS process.xlsx',
 'Report_format.xlsx',
 'Report_format_2.xlsx',
 'Report_format_2_complete.csv',
 'Report_format_2_complete.xlsx',
 'sample_excel.xlsx',
 'sec_10k_analysis_form_filling.xlsx',
 'Series Cover Packer.pptx',
 'Series_Cover_Packer_2.pptx',
 'tasks.xlsx',
 'tasks_with_custom_xaxis.xlsx',
 'template_2.xlsx',
 '_10-K-2021-(As-Filed).pdf']

In [37]:
# Write the report into CSV
with open('./data/Report_format_2_complete.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=headers)
    writer.writeheader()
    for row in flat_data:
        writer.writerow(row)

In [38]:
# Read the CSV and save into Excel File
import pandas as pd
from IPython.core.display import HTML
pd.set_option('display.max_colwidth', 10)
out_df = pd.read_csv("Report_format_2_complete.csv")
out_df.to_excel("./data/Report_format_2_complete.xlsx", index=False)

## Display the 
html = out_df.to_html()
HTML(html)


,Items to be improved,Description,Improvement direction,Person responsible,Expected start date,Actual start date,Expected completion date,Actual completion date,Improve immediately,Confirmation,appendix
0,The saturation of the lower glass point Xiaoli Pill is 65%,Low job saturation(lower than95%),"1. Judgment of value and reduce tasks without added value\n2. Inspection of movement quality and human engineering hazards: Reduce the number of movements, work with both hands at the same time, shorten the distance of movements, and make movements easier; eliminate human engineering hazards\n3. Automated level inspection: simple and automated import\n4. Merge and rearrange new job elements",LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,LiXX,LiXX
1,Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%,Low job saturation(lower than95%),"The improvement direction for the problem 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%' is as follows: 1. Judgment of value and reduce tasks without added value 2. Inspection of movement quality and human engineering hazards: Reduce the number of movements, work with both hands at the same time, shorten the distance of movements, and make movements easier; eliminate human engineering hazards 3. Automated level inspection: simple and automated import 4. Merge and rearrange new job elements.",LiXX,10/8/24,10/8/24,Expected completion date: 10/15/24,10/15/24,yes,The improvement parameter 'Confirmation' for the problem 'Bottom glass electrophoresis tank+UVFixed baking operation saturation79.2%' is 'yes'.,LiXX
2,Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%,Low job saturation(lower than95%),1. Judgment of value and reduce tasks without added value\n2. Inspection of movement quality and human engineering hazards\n3. Automated level inspection\n4. Merge and rearrange new job elements,LiXX,10/8/24,10/8/24,10/15/24,10/15/24,yes,The improvement parameter 'Confirmation' for the problem 'Xiaoliwan wax+Paste conductive foam+Lower glass glue frame dispensing operation saturation75%' is 'yes'.,LiXX


In [39]:
#Reading the exported excel file and develop the  Gantt Chart from there!
import pandas as pd
import plotly.express as px
from openpyxl import load_workbook
from openpyxl.drawing.image import Image
from openpyxl.styles import Alignment

In [ ]:
#Reading the exported excel file and develop the  Gantt Chart from there!
import os
os.getcwd()
os.listdir('./data')
file_path = './data/Report_format_2_complete.xlsx'
df = pd.read_excel(file_path)

# Step 2: Ensure columns are in the correct format
df['Actual start date'] = pd.to_datetime(df['Actual start date'])
df['Actual completion date'] = pd.to_datetime(df['Actual completion date'])

# Step 3: Add a column for data labels (e.g., Task names or durations)
df['Label'] =  (df['Actual completion date'] - df['Actual start date']).dt.days.astype(str)
# df['Label'] = df['Task'] + " (" + ((df['Finish'] - df['Start']).dt.days.astype(str)) + " days)"

# Step 4: Create a Gantt chart with data labels
fig = px.timeline(
    df,
    x_start="Actual start date",
    x_end="Actual completion date",
    y="Items to be improved",
    color="Person responsible",
    text="Label",  # Overlay Task names and durations
    title="Gantt Chart with Custom X-Axis"
)

# Step 5: Customize the x-axis
fig.update_traces(textposition="auto", textfont=dict(size=12, color="white"))

fig.update_yaxes(categoryorder="trace")  # Optional: Sort tasks by total ascending
fig.update_layout(
    xaxis_title="Project Timeline (Dates)",  # Custom x-axis label
    title_x=0.5  # Center the chart title
)

# Step 6: Save the chart as an image
image_file = "./data/gantt_chart_with_custom_xaxis.png"
fig.write_image(image_file)  # Requires kaleido library (install using `pip install kaleido`)

# Step 7: Load the Excel file
wb = load_workbook(file_path)
ws = wb.active  # Use the first sheet (or specify a sheet name)

# Set the width of specific columns
ws.column_dimensions['A'].width = 50  # Set column A width to 50
ws.column_dimensions['B'].width = 40  # Set column A width to 40
ws.column_dimensions['C'].width = 60  # Set column A width to 60
ws.column_dimensions['D'].width = 20  # Set column A width to 20
ws.column_dimensions['E'].width = 20  # Set column A width to 20
ws.column_dimensions['F'].width = 20  # Set column A width to 20
ws.column_dimensions['G'].width = 25  # Set column A width to 25
ws.column_dimensions['H'].width = 25  # Set column A width to 25
ws.column_dimensions['I'].width = 20  # Set column A width to 20
ws.column_dimensions['J'].width = 20  # Set column A width to 20
ws.column_dimensions['K'].width = 20  # Set column A width to 20


# Apply Wrap Text to specific cells
for col in ['A','B','C','D','E','F','G','H','I','J','K']:
    for cell in ws[col]:
        cell.alignment = Alignment(wrap_text=True)


# Step 8: Insert the image into the Excel file
img = Image(image_file)
img.anchor = "M2"  # Position to embed the image (e.g., cell E2)
ws.add_image(img)

# Step 9: Save the updated Excel file
output_file = "./data/tasks_with_custom_xaxis.xlsx"
wb.save(output_file)

print(f"Gantt chart with custom x-axis embedded in {output_file}")


Gantt chart with custom x-axis embedded in ./data/tasks_with_custom_xaxis.xlsx


C:\Users\Al-Amin\AppData\Local\Temp\ipykernel_5764\266155471.py:9: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.

C:\Users\Al-Amin\AppData\Local\Temp\ipykernel_5764\266155471.py:10: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



: 